# Cute pandas 2

Table of Contents

* [Where we left off last](#Where-we-left-off-last)
* [Dtypes](#Dtypes)
* [Converting text to numeric data](#Converting-text-to-numeric-data)
* [NaNs become Zeroes](#NaNs-become-Zeroes)
* [Resources](#Resources)

To run the code cells below either click `Run` in the menu above or `Help` has a list of all the `Keyboard Shortcuts`:
* `Shift + Enter` run the current cell, select below
* `Ctrl + Ente`r run selected cells
* `Alt + Enter` run the current cell, insert below
* `Ctrl + S` save and checkpoint


<strong>Word of Caution.</strong>
<br>Please make sure to run cells in consecutive order. 
Running cells ahead of others will lead to confusion. We are making changes to the same dataframe across this whole notebook. If you run later cells first, you will see those changes immediatly reflected in the dataframe.

## Where we left off last

Here is where we left off at the end of `python_pandas_cleaning_data1.ipynb` file.
In this current file, we will take it a step further and check all the columns data, 
convert it to appropriate formats and ready for analyzing.

In [57]:
# Importing pandas package
import pandas as pd

In [58]:
#Loading csv file with accounting data
fin_sample = pd.read_csv('financial_sample.csv')

# 'Renaming' columns in place by stripping away spaces before and after column names in the existing dataframe
fin_sample.rename(columns=lambda x: x.strip(), inplace=True)

# Stripping spaces around text in dataframe
fin_sample_trimmed = fin_sample.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Stripping away '$-' symbol from dataframe
fin_sample_trimmed_clean = fin_sample_trimmed.apply(lambda x: x.str.strip('$-') if x.dtype == "object" else x)

In [59]:
# At this point point, we are exactly where we left off last...
fin_sample_trimmed_clean.head()

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Government,Canada,Carretera,None,1618.5,3.00,20.00,"32,370.00",,"32,370.00","16,185.00","16,185.00",1/1/14,1,January,2014
1,Government,Germany,Carretera,None,1321.0,3.00,20.00,"26,420.00",,"26,420.00","13,210.00","13,210.00",1/1/14,1,January,2014
2,Midmarket,France,Carretera,None,2178.0,3.00,15.00,"32,670.00",,"32,670.00","21,780.00","10,890.00",6/1/14,6,June,2014
3,Midmarket,Germany,Carretera,None,888.0,3.00,15.00,"13,320.00",,"13,320.00","8,880.00","4,440.00",6/1/14,6,June,2014
4,Midmarket,Mexico,Carretera,None,2470.0,3.00,15.00,"37,050.00",,"37,050.00","24,700.00","12,350.00",6/1/14,6,June,2014


In [60]:
fin_sample_trimmed_clean.columns

Index(['Segment', 'Country', 'Product', 'Discount Band', 'Units Sold',
       'Manufacturing Price', 'Sale Price', 'Gross Sales', 'Discounts',
       'Sales', 'COGS', 'Profit', 'Date', 'Month Number', 'Month Name',
       'Year'],
      dtype='object')

## Dtypes

In [61]:
# What type of data types are we dealing with here? 
# Each column has it's data type assigned/inferred during loading csv data.
# 'Object' data type means in essence a mixed data type. 
# For example 'Country' column could have strings(text) or integers(numbers) for values which is not great.
# We want each column to be a uniform data type so that we can expect how it behaves when we manipulate it
# during analysis. The output below shows us that 'Units Sold' is a float (decimal), 'Month' and 'Year' are integers.
# Every other column is an object which we need to fix.

fin_sample_trimmed_clean.dtypes

Segment                 object
Country                 object
Product                 object
Discount Band           object
Units Sold             float64
Manufacturing Price     object
Sale Price              object
Gross Sales             object
Discounts               object
Sales                   object
COGS                    object
Profit                  object
Date                    object
Month Number             int64
Month Name              object
Year                     int64
dtype: object

In [62]:
# Let's look at 'Sales' column and the first two data points. 
# Both items look like numbers... Are they?

[x for x in fin_sample_trimmed_clean['Sales']][0:2]

['32,370.00', '26,420.00']

In [63]:
# We have strings on our hands... strings in essence mean text data, not numbers or decimals.
[type(x) for x in fin_sample_trimmed_clean['Sales']][0:2]

[str, str]

## Converting text to numeric data

In [64]:
# Each column that currently contains 'numbers' needs to be converted from text(string) to float(decimal)
# as we want to preserve the 0.00 format. Units sold column is alreday a float as we can see above.
# Let's create a list of all of the columns we need to convert.

obj_to_number_columns = ['Manufacturing Price', 'Sale Price', 'Gross Sales', 'Discounts', 'Sales', 'COGS', 'Profit']

In [65]:
# Now let's loop through all the columns and convert each to the proper data type. At the same time,
# we will cast each 'number' to make sure it is a string first and then 'replace' the thousands comma 
# as that will help in the conversion process to floats.
# Here is how we do it!

for column in obj_to_number_columns:
    fin_sample_trimmed_clean[column] = pd.to_numeric(
        fin_sample_trimmed_clean[column].astype(str).str.replace(',',''), errors='coerce'
    )

In [66]:
# Did it work? Yes it did, we have floats.
[type(x) for x in fin_sample_trimmed_clean['Sales']][0:2]

[float, float]

In [67]:
fin_sample_trimmed_clean.dtypes

Segment                 object
Country                 object
Product                 object
Discount Band           object
Units Sold             float64
Manufacturing Price    float64
Sale Price             float64
Gross Sales            float64
Discounts              float64
Sales                  float64
COGS                   float64
Profit                 float64
Date                    object
Month Number             int64
Month Name              object
Year                     int64
dtype: object

## NaNs become Zeroes

In [68]:
# Above, in cell 31  - errors=’coerce’, will transform the non-numeric values into NaN.
# Now we need to convert those to zeroes.

fin_sample_trimmed_clean.head()

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Government,Canada,Carretera,None,1618.5,3.0,20.0,32370.0,NaN,32370.0,16185.0,16185.0,1/1/14,1,January,2014
1,Government,Germany,Carretera,None,1321.0,3.0,20.0,26420.0,NaN,26420.0,13210.0,13210.0,1/1/14,1,January,2014
2,Midmarket,France,Carretera,None,2178.0,3.0,15.0,32670.0,NaN,32670.0,21780.0,10890.0,6/1/14,6,June,2014
3,Midmarket,Germany,Carretera,None,888.0,3.0,15.0,13320.0,NaN,13320.0,8880.0,4440.0,6/1/14,6,June,2014
4,Midmarket,Mexico,Carretera,None,2470.0,3.0,15.0,37050.0,NaN,37050.0,24700.0,12350.0,6/1/14,6,June,2014


In [69]:
# .fillna replaces NaN with zeroes in place - making changes to the existing dataframe
fin_sample_trimmed_clean.fillna(0, inplace=True)

In [70]:
# First five rows
fin_sample_trimmed_clean.head()

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Government,Canada,Carretera,None,1618.5,3.0,20.0,32370.0,0.0,32370.0,16185.0,16185.0,1/1/14,1,January,2014
1,Government,Germany,Carretera,None,1321.0,3.0,20.0,26420.0,0.0,26420.0,13210.0,13210.0,1/1/14,1,January,2014
2,Midmarket,France,Carretera,None,2178.0,3.0,15.0,32670.0,0.0,32670.0,21780.0,10890.0,6/1/14,6,June,2014
3,Midmarket,Germany,Carretera,None,888.0,3.0,15.0,13320.0,0.0,13320.0,8880.0,4440.0,6/1/14,6,June,2014
4,Midmarket,Mexico,Carretera,None,2470.0,3.0,15.0,37050.0,0.0,37050.0,24700.0,12350.0,6/1/14,6,June,2014


In [71]:
# Last five rows
fin_sample_trimmed_clean.tail()

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
20995,Small Business,France,Amarilla,High,2475.0,260.0,300.0,742500.0,111375.00,631125.00,618750.0,12375.00,3/1/2014,3,March,2014
20996,Small Business,Mexico,Amarilla,High,546.0,260.0,300.0,163800.0,24570.00,139230.00,136500.0,2730.00,10/1/2014,10,October,2014
20997,Government,Mexico,Montana,High,1368.0,5.0,7.0,9576.0,1436.40,8139.60,6840.0,1299.60,2/1/2014,2,February,2014
20998,Government,Canada,Paseo,High,723.0,10.0,7.0,5061.0,759.15,4301.85,3615.0,686.85,4/1/2014,4,April,2014
20999,Channel Partners,United States of America,VTT,High,1806.0,250.0,12.0,21672.0,3250.80,18421.20,5418.0,13003.20,5/1/2014,5,May,2014


In [72]:
# Some info on our dataframe
fin_sample_trimmed_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21000 entries, 0 to 20999
Data columns (total 16 columns):
Segment                21000 non-null object
Country                21000 non-null object
Product                21000 non-null object
Discount Band          21000 non-null object
Units Sold             21000 non-null float64
Manufacturing Price    21000 non-null float64
Sale Price             21000 non-null float64
Gross Sales            21000 non-null float64
Discounts              21000 non-null float64
Sales                  21000 non-null float64
COGS                   21000 non-null float64
Profit                 21000 non-null float64
Date                   21000 non-null object
Month Number           21000 non-null int64
Month Name             21000 non-null object
Year                   21000 non-null int64
dtypes: float64(8), int64(2), object(6)
memory usage: 2.6+ MB


In [73]:
# It seems that now we are good to go. Next stop, analysing data.

## Resources

 * [pandas.to_numeric()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_numeric.html?highlight=to_numeric#pandas.to_numeric)
 * [pandas.DataFrame.dtypes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dtypes.html#pandas.DataFrame.dtypes)
 * [pandas.DataFrame.astype()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html#pandas.DataFrame.astype)
 
 
 * [Scroll to Top](#Cute-pandas-2)